In [3]:
pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install smogn

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install resreg

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [7]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import json
from urllib.request import urlopen as urlopen
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score
from glob import glob
from rpy2.robjects import default_converter
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import Converter, localconverter
import rpy2.robjects.numpy2ri
from xgboost import XGBRegressor
import numpy as np
from glob import glob
import os
import itertools as it
import smogn
import resreg


import warnings
warnings.filterwarnings('ignore')

rpy2.robjects.numpy2ri.activate()
pandas2ri.activate()

In [8]:
string = """

U1 <- function(){

  install.packages("devtools")
  library(devtools)

  
  install.packages(c("operators", "class", "fields", "ROCR", "Hmisc", "performanceEstimation"))

  install.packages(c("zoo","xts","quantmod"))

  install.packages( "https://cran.r-project.org/src/contrib/Archive/DMwR/DMwR_0.4.1.tar.gz", repos=NULL, type="source" )

  install_github("nunompmoniz/IRon")
  install_github("paobranco/UBL")
  install_github("rpribeiro/uba")

  library(uba)
  library(UBL)
  library(IRon)
  
}

"""
powerpack = SignatureTranslatedAnonymousPackage(string, "powerpack")

powerpack.U1()

R[write to console]: Loading required package: operators

R[write to console]: 
Attaching package: ‘operators’


R[write to console]: The following objects are masked from ‘package:base’:

    options, strrep


R[write to console]: Loading required package: class

R[write to console]: Loading required package: fields

R[write to console]: Loading required package: spam

R[write to console]: Spam version 2.8-0 (2022-01-05) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

R[write to console]: 
Attaching package: ‘spam’


R[write to console]: The following objects are masked from ‘package:base’:

    backsolve, forwardsolve


R[write to console]: Loading required package: viridis

R[write to console]: Loading required package: viridisLite

R[write to console]: 
Try help(fields) to get started.

R[write to console]

'UBL','randomFo...,'automap',...,'datasets','methods','base'


In [9]:
uba = importr("uba")
iron = importr("IRon")

In [10]:
def scores(y_test, y_pred):
    ph = uba.phi_control(y_test)
    ls = uba.loss_control(y_test)
    prec = uba.util(y_pred, y_test, ph, ls, uba.util_control(umetric="P", event_thr=0.8))
    rec = uba.util(y_pred, y_test, ph, ls, uba.util_control(umetric="R", event_thr=0.8))
    sera = iron.sera(y_test, y_pred, phi_trues = uba.phi(y_test,ph))
    F1 = uba.util(y_pred, y_test, ph, ls, uba.util_control(umetric="Fm", beta=1, event_thr=0.8))

    scores_ = list([mean_squared_error(y_test, y_pred), prec, rec, F1, sera])
    return pd.DataFrame(scores_,
              columns = [''],
              index = ['MSE', 'precision', 'recall', 'fscore', 'sera'])

In [16]:
def balance(train, strategy, c):
  string = """
        
  library(uba)
  library(UBL)
  
  train <- read.csv("/home/jga2/Resultados/train.csv")

  RU <- function(perc){
    resample <- RandUnderRegress(X0~., train, thr.rel=0.8, C.perc=perc)
  }

  RO <- function(perc){
    resample <- RandOverRegress(X0~., train, thr.rel=0.8, C.perc=perc)
  }

  SMT <- function(perc){
    resample <- SmoteRegress(X0~., train, thr.rel=0.8, C.perc=perc)
  }

  GN <- function(perc, pert){
    resample <-GaussNoiseRegress(X0~., train, thr.rel=0.8, C.perc=perc, pert=pert)
  }
  
  """
  powerpack = SignatureTranslatedAnonymousPackage(string, "powerpack")
  if strategy == "GN":
    train = pd.DataFrame(powerpack.GN(c[0], c[1])).T
  elif strategy == "RO":
    train = pd.DataFrame(powerpack.RO(c[0])).T
  elif strategy == "RU":
    train = pd.DataFrame(powerpack.RU(c[0])).T
  elif strategy == "SG":
    train = pd.read_csv("/home/jga2/Resultados/train.csv")
    train = train.dropna(axis=0)        
    train = smogn.smoter(data = train, y = train.columns[0], samp_method=c[0], rel_thres = 0.8)
    train = train.dropna(axis=0)        
  elif strategy == "SMT":
    train = pd.DataFrame(powerpack.SMT(c[0])).T
  elif strategy == "WC":
    train = pd.read_csv("/home/jga2/Resultados/train.csv")
    X_train = train.drop([train.columns[0]], axis = 1)
    y_train  = train[train.columns[0]]
    relevance = resreg.pdf_relevance(y_train)
    X_wercs, y_wercs = resreg.wercs(X_train, y_train, relevance, over=c[0], under=c[1])
    trainWC = np.column_stack((y_wercs, X_wercs))
    pd.DataFrame(trainWC).to_csv("/home/jga2/Resultados/trainWC.csv", index=False)
    train = pd.read_csv("/home/jga2/Resultados/trainWC.csv") 
  return train

In [18]:
def repeatedKfold(n_splits=10, n_repeats=2, random_state=42, pipeline=None, param_grid=None) :
  rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
  all_result = []

  strategys = {"SG":{"samp_method":["balance", "extreme"]},
               "RU":{"C.perc":["balance", "extreme"]},
               "RO":{"C.perc":["balance", "extreme"]},
               "SMT":{"C.perc":["balance", "extreme"]},
               "GN":{"C.perc":["balance", "extreme"], "pert":[0.05, 0.1, 0.5]},
               "WC":{"over":[0.5, 0.8], "under":[0.5, 0.8]},
               'None': {None}}

  for strategy in strategys:

      data_frame = []
      params = strategys[strategy]

      keys = sorted(params)

      if strategy != "None":
        combinations = it.product(*(params[Name] for Name in keys))
      else:
        combinations = ['None']
      for c in list(combinations):
        score_perc = []
        for train_index, test_index in rkf.split(X, y):
          
          X_train, X_test = X[train_index], X[test_index]
          y_train, y_test = y[train_index], y[test_index]
          
          train = np.column_stack((y_train, X_train))
          pd.DataFrame(train).to_csv("/home/jga2/Resultados/train.csv", index=False)
          train = pd.read_csv("/home/jga2/Resultados/train.csv")
          
          if c != 'None':
            train = balance(train, strategy, c)
          
          X_train = train.drop([train.columns[0]], axis = 1)
          y_train  = train[train.columns[0]]

          X_train = X_train.to_numpy()
          y_train = y_train.to_numpy()
          
          grid_search = GridSearchCV(pipeline, cv=rkf, param_grid=param_grid)
          grid_search.fit(X_train, y_train)
          y_pred  = grid_search.predict(X_test)

          path = dataset
          head, tail = os.path.split(path)

          test = np.column_stack((test_index, y_test))
          pred = np.column_stack((test_index, y_pred))
        

          score_perc.append(scores(y_test, y_pred).T) 

        df = pd.concat(score_perc)
        values = [tail, 
                  str(df.precision.mean().round(3)[0])+ "({})".format(df.precision.std().round(3)),
                  str(df.recall.mean().round(3)[0])+ "({})".format(df.recall.std().round(3)),
                  str(df.MSE.mean().round(3))+ "({})".format(df.MSE.std().round(3)),
                  str(df.fscore.mean().round(3)[0])+ "({})".format(df.fscore.std().round(3)),
                  str(df.sera.mean().round(3)[0])+ "({})".format(df.sera.std().round(3))]

        scores_df = pd.DataFrame([values], columns=["Dataset", "Precision", "Recall", "MSE", "Fscore", "SERA"])

        if len(keys) > 1:
          scores_df[keys[0]]=c[0]
          scores_df[keys[1]]=c[1]
          scores_df['strategy']=strategy           
        else:
          scores_df[keys[0]]=c[0]
          scores_df['strategy']=strategy

        data_frame.append(scores_df)
      data_frame = pd.concat(data_frame)
      data_frame.to_csv('/home/jga2/Resultados/result_{}_{}.csv'.format(strategy, str(pipeline.steps[0][1]).split('(')[0]), index = False)
      all_result.append(data_frame)
  return all_result

In [19]:
def pipe_generation():
  clf_param = dict()
  for clf in [BaggingRegressor(DecisionTreeRegressor()), DecisionTreeRegressor(), MLPRegressor(max_iter=200), RandomForestRegressor(), SVR(), XGBRegressor(verbosity=0)]:
      clf_param[str(clf).split('(')[0]] = clf

  pipes_params = []
  for clf,  param_grid in zip([BaggingRegressor(DecisionTreeRegressor()), DecisionTreeRegressor(), MLPRegressor(max_iter=200), RandomForestRegressor(), SVR(), XGBRegressor(verbosity=0)],
                               
                  [{'clf__base_estimator__min_samples_split': [20], 'clf__max_samples':[0.5]},
                   {'clf__min_samples_split': [20]},
                   {'clf__learning_rate_init': [0.1],'clf__momentum': (0.2, 0.7),'clf__tol': (0.01, 0.05)},
                   {'clf__n_estimators': [550, 1500], 'clf__max_features': [5]},
                   {'clf__gamma': [0.01, 0.001], 'clf__C': [10, 300]},
                   {'clf__eta': [0.01], 'clf__max_depth': (10, 15), 'clf__colsample_bytree': (0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9), 'clf__num_round': [25]}]):

    configs = []
    clf = str(clf).split('(')[0]
    for p in param_grid:
        aux = p
        for i in param_grid[p]:
          aux += '+'+str(i)
        clf += '|'+aux
    configs.append(clf)
  
    for config in configs:
    
      pipeline = Pipeline([('clf', clf_param[config.split('|')[0]])])
      params = config.split('|')
    
      param_grid = {}
      t, t1 = len(params), 0
      for p in range(len(params)):
        values = ()
        if len(params[p].split('+')) > 2:
          a = params[p].split('+')[1:]
          for j in a:
            if '0.' in j:
              values += (float(j),)
            else:
              values += (int(j),)
          
          param_grid[params[p].split('+')[0]] = values
        
        else:
    
          if t1 == t:   
            if '0.' in params[p].split('+')[1]:
              param_grid[params[p].split('+')[0]] = [params[p].split('+')[1]]
            else:
              param_grid[params[p].split('+')[0]] = [params[p].split('+')[1]]
          elif t1 < t:
            for l in params[t1].split('+')[1:]:

              if '0.' in l:
                param_grid[params[t1].split('+')[0]] = [float(l)]
              else:
                param_grid[params[t1].split('+')[0]] = [int(l)]

        t1 += 1
  
    pipes_params.append([pipeline, param_grid]) 
  return pipes_params

In [14]:
data_sets = sorted(glob(r'/home/jga2/Downloads/data/a1.csv'))

In [20]:
score_ap = []
for i, dataset in enumerate(data_sets):
  ds = pd.read_csv(dataset)

  path = dataset
  head, tail = os.path.split(path)
  print("=====================")
  
  X = ds.drop(['Unnamed: 0', ds.columns[1]], axis = 1)
  y = ds[ds.columns[1]]

  X = X.to_numpy()
  y = y.to_numpy()

  pipes_params = pipe_generation()
  for j in pipes_params:
    pipeline, param_grid = j[0], j[1]
    print(str(pipeline.steps[0][1]).split('(')[0])
    score_all = pd.concat(repeatedKfold(pipeline=pipeline, param_grid=param_grid))
    print(score_all)

dist_matrix:  56%|#####6    | 14/25 [00:00<00:00, 137.26it/s]

BaggingRegressor


dist_matrix:  56%|#####6    | 14/25 [00:00<00:00, 137.34it/s]

    C.perc Dataset        Fscore               MSE     Precision  \
0      NaN  a1.csv   0.42(0.326)  416.078(160.343)  0.382(0.307)   
0      NaN  a1.csv  0.412(0.329)   366.823(142.06)  0.373(0.304)   
0  balance  a1.csv  0.369(0.361)  544.702(181.721)  0.346(0.351)   
0  extreme  a1.csv  0.298(0.254)  1879.202(341.96)  0.224(0.191)   
0  balance  a1.csv   0.42(0.331)  370.087(148.084)   0.385(0.31)   
0  extreme  a1.csv  0.399(0.316)  419.247(132.893)  0.365(0.294)   
0  balance  a1.csv  0.443(0.346)  349.273(141.206)  0.413(0.329)   
0  extreme  a1.csv   0.397(0.31)  658.289(187.763)  0.333(0.268)   
0  balance  a1.csv   0.43(0.343)  376.215(135.387)  0.401(0.336)   
0  balance  a1.csv  0.441(0.344)   351.153(128.11)  0.409(0.325)   
0  balance  a1.csv  0.428(0.336)  389.764(143.381)  0.393(0.315)   
0  extreme  a1.csv   0.373(0.29)  782.165(268.872)   0.31(0.248)   
0  extreme  a1.csv  0.395(0.305)  750.531(219.092)    0.33(0.26)   
0  extreme  a1.csv  0.383(0.297)  768.505(207.01

dist_matrix:  52%|#####2    | 13/25 [00:00<00:00, 127.12it/s]

...

    C.perc Dataset        Fscore                MSE     Precision  \
0      NaN  a1.csv  0.423(0.334)   364.868(127.299)   0.391(0.32)   
0      NaN  a1.csv   0.424(0.34)   306.816(115.835)  0.409(0.346)   
0  balance  a1.csv  0.434(0.342)   436.262(187.371)  0.388(0.317)   
0  extreme  a1.csv  0.363(0.282)  1229.194(395.147)  0.289(0.231)   
0  balance  a1.csv  0.412(0.329)   288.066(122.037)  0.393(0.323)   
0  extreme  a1.csv  0.417(0.334)   280.082(111.322)  0.408(0.342)   
0  balance  a1.csv  0.425(0.337)   317.892(117.486)  0.396(0.324)   
0  extreme  a1.csv    0.41(0.32)   547.231(192.251)  0.354(0.285)   
0  balance  a1.csv  0.438(0.345)   317.292(116.881)  0.411(0.329)   
0  balance  a1.csv  0.443(0.346)    326.376(126.32)  0.418(0.334)   
0  balance  a1.csv  0.434(0.342)   321.881(130.253)  0.404(0.326)   
0  extreme  a1.csv  0.408(0.321)   570.638(185.911)  0.359(0.294)   
0  extreme  a1.csv  0.414(0.323)   603.185(208.338)  0.361(0.294)   
0  extreme  a1.csv  0.395(0.305)  

dist_matrix:  56%|#####6    | 14/25 [00:00<00:00, 138.36it/s]

    C.perc Dataset        Fscore                MSE     Precision  \
0      NaN  a1.csv  0.412(0.334)   454.725(149.255)  0.426(0.366)   
0      NaN  a1.csv  0.344(0.311)   411.462(162.297)  0.343(0.316)   
0  balance  a1.csv  0.426(0.338)   393.551(155.104)    0.41(0.33)   
0  extreme  a1.csv  0.343(0.269)  1788.856(398.539)  0.264(0.216)   
0  balance  a1.csv  0.329(0.324)   411.569(178.465)   0.35(0.365)   
0  extreme  a1.csv  0.326(0.321)   414.754(177.768)  0.346(0.364)   
0  balance  a1.csv    0.37(0.34)   433.396(194.012)   0.35(0.334)   
0  extreme  a1.csv  0.395(0.321)    865.817(255.03)  0.343(0.288)   
0  balance  a1.csv  0.407(0.328)   416.304(157.917)  0.393(0.324)   
0  balance  a1.csv  0.423(0.333)   402.159(192.587)   0.41(0.328)   
0  balance  a1.csv  0.386(0.336)   442.828(226.689)  0.373(0.332)   
0  extreme  a1.csv  0.396(0.326)    700.292(247.93)   0.37(0.315)   
0  extreme  a1.csv  0.421(0.339)   710.861(230.332)  0.388(0.322)   
0  extreme  a1.csv   0.336(0.26)  

r_index: 100%|##########| 23/23 [00:00<00:00, 683.01it/s]


    C.perc Dataset        Fscore               MSE     Precision  \
0      NaN  a1.csv   0.17(0.252)  317.119(138.558)  0.174(0.264)   
0      NaN  a1.csv  0.179(0.277)  323.669(154.576)  0.197(0.313)   
0  balance  a1.csv  0.161(0.261)  297.414(126.927)   0.165(0.27)   
0  extreme  a1.csv  0.185(0.262)    443.02(131.03)  0.162(0.228)   
0  balance  a1.csv  0.169(0.255)  341.486(167.907)  0.178(0.268)   
0  extreme  a1.csv  0.159(0.259)  330.142(157.146)  0.175(0.281)   
0  balance  a1.csv  0.186(0.279)  317.999(138.869)  0.204(0.311)   
0  extreme  a1.csv  0.216(0.283)  298.416(104.451)  0.219(0.292)   
0  balance  a1.csv  0.145(0.242)  301.571(140.134)  0.145(0.244)   
0  balance  a1.csv  0.152(0.253)  303.124(149.618)  0.162(0.276)   
0  balance  a1.csv  0.163(0.267)  297.397(131.233)  0.173(0.286)   
0  extreme  a1.csv  0.259(0.309)  322.396(134.759)  0.255(0.315)   
0  extreme  a1.csv  0.224(0.298)  302.315(114.575)  0.221(0.302)   
0  extreme  a1.csv  0.229(0.303)   310.091(100.8